In [ ]:
import sys
!python -m pip install ..

In [ ]:
import pickle
import pandas as pd
import adad
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, RocCurveDisplay
import os
import adad
from adad.utils import to_json, open_json
from sklearn.ensemble import RandomForestClassifier
from adad.distance import DAIndexGamma
import numpy as np
from adad.evaluate import sensitivity_specificity, cumulative_accuracy, roc_ad, permutation_auc, predictiveness_curves
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## Functions

In [ ]:
def trainCLF(dataset, train_split, clf, name, path=None, scale=True):
    if path is None:
        path = os.getcwd()
    
    #Get train and test
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    if scale:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    #Train classifier
    clf.fit(X_train, y_train)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    
    print(f"Accuracy evaluation of {type(clf).__name__} for {name} dataset:")
    print("===============================================================")
    print(f"Accuracy of training split: {accuracy_score(y_train, train_pred):.3f}")
    print(f"Accuracy of testing split : {accuracy_score(y_test, test_pred):.3f}\n")
    
    #Save classifier
    filename = f"{type(clf).__name__ }_{name}"
    with open(os.path.join(path, filename + ".pickle"), 'wb') as file:
        pickle.dump(clf, file)
    
    #Get JSON file
    parameter_json = clf.get_params()
    to_json(parameter_json, os.path.join(path, filename + ".json"))
    
    json_file = open_json(os.path.join(path, filename + ".json"))
    return json_file

### Stores all evaluations for all cv into one DataFrame

In [ ]:
def runAD(dataset, cv_split, clf, ad, name, path=None, scale=True):
    #set up ad, scaler, and path
    ad.clf = clf
    scaler = StandardScaler()
    if path == None:
        path = os.getcwd()
    
    #Get X and y
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    size = X.shape[0]
    
    #Prepare DataFrames for evaluation and score data
    evaluation = pd.DataFrame(columns=['sensitivity', 'specificity', 'cumulative_acc', 'roc_ad', 'auc(roc_ad)','permutation_auc', 
                                       'permutation_list', 'predictiveness_curves'])
    measure = pd.DataFrame()
    
    #Test ad for all cv
    for col in cv_split:
        #Find indexes
        train_idx = cv_split[col].to_numpy()
        train_idx = train_idx[~np.isnan(train_idx)].astype(int)
        
        all_idx = np.arange(0, size)
        bool_train = np.isin(all_idx, train_idx)
        test_idx = all_idx[~bool_train]
        
        #Scale train and test datasets
        X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
        if scale:
            
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
        
        #Train AD and classifier
        ad.clf.fit(X_train, y_train)
        ad.fit(X_train)
        
        #Gather scores and save them in csv
        dist_measure = ad.measure(X_test)
        new_col = pd.DataFrame(np.around(dist_measure, decimals=6), columns=[f"cv_{col+1}"])
        measure = pd.concat([measure, new_col], axis=1)
        
        #Start gathering data from evaluation functions
        y_pred = ad.clf.predict(X_test)
        
        sensitivity, specificity = sensitivity_specificity(y_test, y_pred)
        cumulative_acc, cumulative_rate = cumulative_accuracy(y_test, y_pred, dist_measure)
        fpr, tpr = roc_ad(y_test, y_pred, dist_measure)
        auc_signi, auc_perm = permutation_auc(y_test, y_pred, dist_measure)
        percentile, err_rate = predictiveness_curves(y_test, y_pred, dist_measure)
        auc_roc = auc(fpr, tpr)
        
        #Compare AUCs
        print(f"permutationAUC vs auc(roc_ad) of {name} for split{col+1}:")
        print("==================================================================")
        print(f"         {auc_signi:.3f} vs {auc_roc:.3f}\n")
        
        
        #Create graphs to save for each split
        fig, axes = plt.subplots(2,2,figsize=(6,6))
        axes[0, 0].plot(cumulative_rate, cumulative_acc)
        axes[0, 0].set_title("Cumulative Accuracy")
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr,roc_auc=auc_roc)
        roc_display.plot(ax=axes[0, 1])
        axes[0, 1].set_title("AUC ROC")
        axes[1, 0].plot(percentile, err_rate)
        axes[1, 0].set_title("Predictiveness Curves (PC)")
        plt.setp(axes[0, 0], xlabel='Cumulative Rate', ylabel='Cumulative Accuracy (%)')
        plt.setp(axes[1, 0], xlabel='Percentile', ylabel='Error Rate')
        fig.delaxes(axes[1, 1])
        fig.suptitle(f'Graphs of {name} for split{col+1}', fontsize=16)
        fig.tight_layout()
        fig.savefig(f'{os.path.join(path, f"{name}_split{col+1}_graphs.png")}', dpi=300)
        
        #Save evaluation
        value_dict = {'sensitivity': np.round(sensitivity, 6), 'specificity': np.round(specificity, 6), 
                      'cumulative_acc': list((np.around(cumulative_acc, 6), np.around(cumulative_rate, 6))),
                      'roc_ad': list((np.around(fpr, 6), np.around(tpr, 6))), 'auc(roc_ad)': auc_roc, 
                      'permutation_auc': auc_signi, 'permutation_list': [auc_perm],
                      'predictiveness_curves': list((np.around(percentile, 6), np.around(err_rate, 6)))}
        
        evaluation = evaluation.append(value_dict, ignore_index=True)
    
    measure.to_csv(os.path.join(path, f'{name}_scores.csv'), index=False)
    evaluation.to_csv(os.path.join(path, f'{name}_evaluation.csv'), index=False)
    
    return evaluation
        

### One DataFrame for each cv evaluation

In [ ]:
def runAD(dataset, cv_split, clf, ad, name, path=None, scale=True):
    #set up ad, scaler, and path
    ad.clf = clf
    scaler = StandardScaler()
    if path == None:
        path = os.getcwd()
    
    #Get X and y
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    size = X.shape[0]
    
    #Prepare DataFrames for evaluation and score data
    measure = pd.DataFrame()
    evaluation = pd.DataFrame()
    evaluations = []
    
    #Test ad for all cv
    for col in cv_split:
        #Find indexes
        train_idx = cv_split[col].to_numpy()
        train_idx = train_idx[~np.isnan(train_idx)].astype(int)
        
        all_idx = np.arange(0, size)
        bool_train = np.isin(all_idx, train_idx)
        test_idx = all_idx[~bool_train]
        
        #Scale train and test datasets
        X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
        if scale:
            
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
        
        #Train AD and classifier
        ad.clf.fit(X_train, y_train)
        ad.fit(X_train)
        
        #Gather scores and save them in csv
        dist_measure = ad.measure(X_test)
        new_col = pd.DataFrame(np.around(dist_measure, decimals=6), columns=[f"cv_{col+1}"])
        measure = pd.concat([measure, new_col], axis=1)
        
        #Start gathering data from evaluation functions
        y_pred = ad.clf.predict(X_test)
        
        sensitivity, specificity = sensitivity_specificity(y_test, y_pred)
        cumulative_acc, cumulative_rate = cumulative_accuracy(y_test, y_pred, dist_measure)
        fpr, tpr = roc_ad(y_test, y_pred, dist_measure)
        auc_signi, auc_perm = permutation_auc(y_test, y_pred, dist_measure)
        percentile, err_rate = predictiveness_curves(y_test, y_pred, dist_measure)
        auc_roc = auc(fpr, tpr)
        
        #Compare AUCs
        print(f"permutationAUC vs auc(roc_ad) of {name} for split{col+1}:")
        print("==================================================================")
        print(f"         {auc_signi:.3f} vs {auc_roc:.3f}\n")
        
        
        #Create graphs to save for each split
        fig, axes = plt.subplots(2,2,figsize=(6,6))
        axes[0, 0].plot(cumulative_rate, cumulative_acc)
        axes[0, 0].set_title("Cumulative Accuracy")
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr,roc_auc=auc_roc)
        roc_display.plot(ax=axes[0, 1])
        axes[0, 1].set_title("AUC ROC")
        axes[1, 0].plot(percentile, err_rate)
        axes[1, 0].set_title("Predictiveness Curves (PC)")
        plt.setp(axes[0, 0], xlabel='Cumulative Rate', ylabel='Cumulative Accuracy (%)')
        plt.setp(axes[1, 0], xlabel='Percentile', ylabel='Error Rate')
        fig.delaxes(axes[1, 1])
        fig.suptitle(f'Graphs of {name} for split{col+1}', fontsize=16)
        fig.tight_layout()
        fig.savefig(f'{os.path.join(path, f"{name}_split{col+1}_graphs.png")}', dpi=300)
        
        #Save evaluation
        column1 = pd.DataFrame([np.round(sensitivity, 6)], columns=['sensitivity'])
        column2 = pd.DataFrame([np.round(specificity, 6)],columns=['specificity'])
        column3 = pd.DataFrame(np.around(cumulative_acc, 6),columns=['cumulative_acc'])
        column4 = pd.DataFrame(np.around(cumulative_rate, 6), columns=['cumulative_rate'])
        column5 = pd.DataFrame(np.around(fpr, 6),columns=['roc_ad_fpr'])
        column6 = pd.DataFrame(np.around(tpr, 6),columns=['roc_ad_tpr'])
        column7 = pd.DataFrame([np.round(auc_roc,6)] ,columns=['auc(roc_ad)'])
        column8 = pd.DataFrame([np.round(auc_signi, 6)],columns=['permutation_auc'])
        column9 = pd.DataFrame(np.around(auc_perm,6),columns=['permutation_list'])
        column10 = pd.DataFrame(np.around(percentile, 6),columns=['pred_curve_percent'])
        column11 = pd.DataFrame(np.around(err_rate, 6),columns=['pred_curve_err'])
        
        evaluation = pd.concat([column1, column2, column3, column4, column5, column6, column7, column8, column9, column10, column11],axis=1)
        evaluation.to_csv(os.path.join(path, f'{name}_cv{col+1}_evaluation.csv'), index=False)
        evaluations.append(evaluation)
    
    measure.to_csv(os.path.join(path, f'{name}_scores.csv'), index=False)
    
    return evaluations

### One DataFrame per Evaluation

In [ ]:
#TODO
def runAD(dataset, cv_split, clf, ad, name, path=None, scale=True):
    #set up ad, scaler, and path
    ad.clf = clf
    scaler = StandardScaler()
    if path == None:
        path = os.getcwd()
    
    #Get X and y
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    size = X.shape[0]
    
    #Prepare DataFrames for evaluation and score data
    measure = pd.DataFrame()
    single_value = pd.DataFrame()
    
    evaluations = []
    
    #Test ad for all cv
    for col in cv_split:
        #Find indexes
        train_idx = cv_split[col].to_numpy()
        train_idx = train_idx[~np.isnan(train_idx)].astype(int)
        
        all_idx = np.arange(0, size)
        bool_train = np.isin(all_idx, train_idx)
        test_idx = all_idx[~bool_train]
        
        #Scale train and test datasets
        X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
        if scale:
            
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
        
        #Train AD and classifier
        ad.clf.fit(X_train, y_train)
        ad.fit(X_train)
        
        #Gather scores and save them in csv
        dist_measure = ad.measure(X_test)
        new_col = pd.DataFrame(np.around(dist_measure, decimals=6), columns=[f"cv_{col+1}"])
        measure = pd.concat([measure, new_col], axis=1)
        
        #Start gathering data from evaluation functions
        y_pred = ad.clf.predict(X_test)
        
        sensitivity, specificity = sensitivity_specificity(y_test, y_pred)
        cumulative_acc, cumulative_rate = cumulative_accuracy(y_test, y_pred, dist_measure)
        fpr, tpr = roc_ad(y_test, y_pred, dist_measure)
        auc_signi, auc_perm = permutation_auc(y_test, y_pred, dist_measure)
        percentile, err_rate = predictiveness_curves(y_test, y_pred, dist_measure)
        auc_roc = auc(fpr, tpr)
        
        #Compare AUCs
        print(f"permutationAUC vs auc(roc_ad) of {name} for split{col+1}:")
        print("==================================================================")
        print(f"         {auc_signi:.3f} vs {auc_roc:.3f}\n")
        
        
        #Create graphs to save for each split
        fig, axes = plt.subplots(2,2,figsize=(6,6))
        axes[0, 0].plot(cumulative_rate, cumulative_acc)
        axes[0, 0].set_title("Cumulative Accuracy")
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr,roc_auc=auc_roc)
        roc_display.plot(ax=axes[0, 1])
        axes[0, 1].set_title("AUC ROC")
        axes[1, 0].plot(percentile, err_rate)
        axes[1, 0].set_title("Predictiveness Curves (PC)")
        plt.setp(axes[0, 0], xlabel='Cumulative Rate', ylabel='Cumulative Accuracy (%)')
        plt.setp(axes[1, 0], xlabel='Percentile', ylabel='Error Rate')
        fig.delaxes(axes[1, 1])
        fig.suptitle(f'Graphs of {name} for split{col+1}', fontsize=16)
        fig.tight_layout()
        fig.savefig(f'{os.path.join(path, f"{name}_split{col+1}_graphs.png")}', dpi=300)
        
        #Save evaluation
        column1 = pd.DataFrame([np.round(sensitivity, 6)], columns=['sensitivity'])
        column2 = pd.DataFrame([np.round(specificity, 6)],columns=['specificity'])
        column3 = pd.DataFrame(np.around(cumulative_acc, 6),columns=['cumulative_acc'])
        column4 = pd.DataFrame(np.around(cumulative_rate, 6), columns=['cumulative_rate'])
        column5 = pd.DataFrame(np.around(fpr, 6),columns=['roc_ad_fpr'])
        column6 = pd.DataFrame(np.around(tpr, 6),columns=['roc_ad_tpr'])
        column7 = pd.DataFrame([np.round(auc_roc,6)] ,columns=['auc(roc_ad)'])
        column8 = pd.DataFrame([np.round(auc_signi, 6)],columns=['permutation_auc'])
        column9 = pd.DataFrame(np.around(auc_perm,6),columns=['permutation_list'])
        column10 = pd.DataFrame(np.around(percentile, 6),columns=['pred_curve_percent'])
        column11 = pd.DataFrame(np.around(err_rate, 6),columns=['pred_curve_err'])
        
        evaluation = pd.concat([column1, column2, column3, column4, column5, column6, column7, column8, column9, column10, column11],axis=1)
        evaluation.to_csv(os.path.join(path, f'{name}_cv{col+1}_evaluation.csv'), index=False)
        evaluations.append(evaluation)
    
    measure.to_csv(os.path.join(path, f'{name}_scores.csv'), index=False)
    
    return evaluations

## Demo

In [ ]:
path = os.getcwd()
parent_path = os.path.abspath(os.path.join(path, os.pardir))

files_path = os.path.join(parent_path, 'data', 'maccs')
dataset_files = [os.path.join(files_path, file) for file in os.listdir(files_path)]

cv_path = os.path.join(parent_path, 'experiments', 'preprocessing')
cv_files = [os.path.join(cv_path, file) for file in os.listdir(cv_path)]

dataset = pd.read_csv(filename)
cv_split = pd.read_csv(cv, header=None, dtype={'id': int})

results_path = os.path.join(parent_path, 'results')

### Select dataset by Index

    Ames      |    0
    BBBP      |    1
    Cancer    |    2
    CYP1A2    |    3
    FXa       |    4
    hERG      |    5
    HIV       |    6
    Liver     |    7

In [ ]:
index = 3
names_list = ['Ames', 'BBBP', 'Cancer', 'CYP1A2', 'FXa', 'hERG', 'HIV', 'Liver']
filename = dataset_files[index]
cv = cv_files[index]
name = names_list[index]

In [ ]:
SEED = 42
rfc = RandomForestClassifier(n_estimators=300, random_state=SEED)

In [ ]:
json = trainCLF(dataset, cv_split, rfc, name, path=results_path)

In [ ]:
print(json)

In [ ]:
ad = DAIndexGamma(clf=rfc)
evaluation = runAD(dataset, cv_split, rfc, ad, name+"s", path=results_path)

## Using train_test_split

In [ ]:
def testing(dataset, cv_split, clf, ad, name, path=None, scale=True):
    if path is None:
        path = os.getcwd()
    
    #Get train and test
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if scale:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    #Train classifier
    clf.fit(X_train, y_train)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    
    print(f"Accuracy evaluation of {type(clf).__name__} for {name} dataset:")
    print("===============================================================")
    print(f"Accuracy of training split: {accuracy_score(y_train, train_pred):.3f}")
    print(f"Accuracy of testing split : {accuracy_score(y_test, test_pred):.3f}\n")
    
    ad.fit(X_train)
        
    #Gather scores and save them in csv
    dist_measure = ad.measure(X_test)
    new_col = pd.DataFrame(dist_measure)

    #Start gathering data from evaluation functions
    y_pred = clf.predict(X_test)

    sensitivity, specificity = sensitivity_specificity(y_test, y_pred)
    cumulative_acc, cumulative_rate = cumulative_accuracy(y_test, y_pred, dist_measure)
    fpr, tpr = roc_ad(y_test, y_pred, dist_measure)
    auc_signi, auc_perm = permutation_auc(y_test, y_pred, dist_measure)
    percentile, err_rate = predictiveness_curves(y_test, y_pred, dist_measure)
    auc_roc = auc(fpr, tpr)

    #Compare AUCs
    print(f"permutationAUC vs auc(roc_ad) of {name}:")
    print(f"         {auc_signi:.3f} vs {auc_roc:.3f}\n")


    #Create graphs to save for each split
    fig, axes = plt.subplots(2,2,figsize=(6,6))
    axes[0, 0].plot(cumulative_rate, cumulative_acc)
    axes[0, 0].set_title("Cumulative Accuracy")
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr,roc_auc=auc_roc)
    roc_display.plot(ax=axes[0, 1])
    axes[0, 1].set_title("AUC ROC")
    axes[1, 0].plot(percentile, err_rate)
    axes[1, 0].set_title("Predictiveness Curves (PC)")
    plt.setp(axes[0, 0], xlabel='Cumulative Rate', ylabel='Cumulative Accuracy (%)')
    plt.setp(axes[1, 0], xlabel='Percentile', ylabel='Error Rate')
    fig.delaxes(axes[1, 1])
    fig.suptitle(f'Graphs of {name}', fontsize=16)
    fig.tight_layout()
    fig.savefig(f'{os.path.join(path, f"{name}_graphs.png")}', dpi=300)

In [ ]:
testing(dataset, cv_split, rfc, ad, name, path=results_path)